## 1. Vertex mapping (mm + z-shift)

- Compare to the ROOT source: `EvtVtx[0..2]` (meters) × **1000** and then **add** `G4_ZSHIFT_MM` to $z$.
- Pass if $|\Delta x|$, $|\Delta y|$, $|\Delta z| \lesssim$ **1–2 mm** (we allow ±1–2 mm for print precision).

## 2. Inside the can

- Compute $r = \sqrt{x^2 + y^2}$ and check $r < R_{\text{can}}^{\text{mm}}$ and $|z| < Z_{\text{half}}^{\text{mm}}$ (from the geometry).
- Pass if **all 10** are inside.

## 3. Direction (unit vector $\vec{u}$)

- From ROOT, take the muon 3-momentum $(p_x, p_y, p_z)$ in **GeV** → convert to **MeV** (×1000) and make it unit length.
- Compute $\Delta\theta = \arccos(\vec{u} \cdot \vec{u}_{\text{ref}})$.
- Pass if $\Delta\theta \leq$ **1.0°** for each event.

## 4. Energy (MeV)

- From ROOT, $E$ (GeV) ×1000 ⇒ compare to the printed $E$ (MeV).
- Pass if $\frac{|E_{\text{g4}} - E_{\text{root}}|}{E_{\text{root}}} \leq$ **1%**.
- Bonus sanity: for a muon, check $E^2 - |\vec{p}|^2 \approx m_\mu^2 \approx (105.658 \text{ MeV})^2$.

## 5. Time (ns)

- If propagate `EvtVtx[3]` (seconds) → **ns**, the printed $t$ (ns) should match within ~1 ns.

In [ ]:
import os
# Make sure the notebook applies the same shift the generator used
os.environ["G4_ZSHIFT_MM"] = "-20000"

# Also tighten the can half-length to the actual value from GDML:
# tube z = 4000 cm → half-length = 2000 cm = 20000 mm
R_CAN_MM  = 1500.0
Z_HALF_MM = 20000.0


In [7]:
# Hand-check: vertex (mm+zshift), in-can, Δθ, ΔE/E, Δt, E, T.

import os, re, math, numpy as np, pandas as pd, uproot

# ------------------ CONFIG ------------------
LOG_PATH  = os.getenv("FLN_HC_LOG",
    "/Users/jingyuanzhang/Desktop/AstroParticle/FLOUNDER/Simulations/GEANT4data/docs/day1/handcheck.log")
ROOT_PATH = os.getenv("G4_ROOTRACKER",
    "/Users/jingyuanzhang/Desktop/AstroParticle/FLOUNDER/Simulations/Data/hand_off/handoff_pilot_muCC_200.gtrac.root")

# geometry (from your GDML: rmax=150 cm → 1500 mm, half-length=2000 cm → 20000 mm)
R_CAN_MM  = float(os.getenv("FLN_CAN_R_MM", 1500))
Z_HALF_MM = float(os.getenv("FLN_CAN_ZHALF_MM", 20000))

# z-shift used in GEANT4 primaries (must match GEANT4.sh)
ZSHIFT_MM = float(os.getenv("G4_ZSHIFT_MM", -20000))

# Acceptance thresholds
TOL_VTX_MM = 2.0
TOL_ANG_DEG = 1.0
TOL_ERE_PCT = 1.0
TOL_DT_NS   = 1.0

MU_MEV = 105.658  # PDG muon mass

# ------------------ Parse GEANT4 log ------------------
num = r'[-+]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][-+]?\d+)?'
pat = re.compile(
    rf"evt\s*=\s*(\d+).*?vtx\(mm\)\s*=\s*\(\s*({num})\s*,\s*({num})\s*,\s*({num})\s*\)"
    rf"(?:.*?t\(ns\)\s*=\s*({num}))?"
    rf".*?p\(MeV\)\s*=\s*\(\s*({num})\s*,\s*({num})\s*,\s*({num})\s*\)"
    rf".*?\|p\|\s*=\s*({num}).*?u\s*=\s*\(\s*({num})\s*,\s*({num})\s*,\s*({num})\s*\)"
    rf"(?:.*?E\(MeV\)\s*=\s*({num}))?",  # optional if your print includes E
    re.IGNORECASE
)

g4 = {}
with open(LOG_PATH, "r", errors="ignore") as f:
    for line in f:
        m = pat.search(line)
        if not m: 
            continue
        ev = int(m.group(1))
        g4[ev] = dict(
            x=float(m.group(2)),  y=float(m.group(3)),  z=float(m.group(4)),
            t=float(m.group(5)) if m.group(5) not in (None, "") else np.nan,
            px=float(m.group(6)), py=float(m.group(7)), pz=float(m.group(8)),
            p=float(m.group(9)),
            ux=float(m.group(10)), uy=float(m.group(11)), uz=float(m.group(12)),
            E_log=float(m.group(13)) if m.group(13) not in (None, "") else np.nan  # from log if present
        )

if not g4:
    raise SystemExit(f"[ERROR] Parsed 0 events from {LOG_PATH}. "
                     "Re-run the 10-event loop and ensure lines contain 'vtx(mm)='.")

# ------------------ Load ROOT truth ------------------
with uproot.open(ROOT_PATH) as rf:
    t   = rf["gRooTracker"]
    vtx = t["EvtVtx"].array(library="np")          # (x[m], y[m], z[m], t[s])
    pdg = t["StdHepPdg"].array(library="np")
    p4  = t["StdHepP4"].array(library="np")        # (px,py,pz,E) in GeV

# pick first muon (±13)
mu_idx = {}
for i in range(len(pdg)):
    idx = np.where(np.abs(pdg[i]) == 13)[0]
    mu_idx[i] = int(idx[0]) if idx.size else None

def unit(v):
    n = np.linalg.norm(v)
    return v/n if n>0 else v*0

# ------------------ Compare & build table ------------------
rows = []
for ev in sorted(g4.keys()):
    rec = g4[ev]

    # GEANT4 (measured)
    xg, yg, zg = rec["x"], rec["y"], rec["z"]
    t_g4_ns    = rec["t"]
    p_mag_mev  = rec["p"]
    u_g4       = np.array([rec["ux"], rec["uy"], rec["uz"]], dtype=float)
    # Energy from |p| and mu mass (robust even if log didn't print E)
    E_g4_mev   = math.sqrt(p_mag_mev*p_mag_mev + MU_MEV*MU_MEV)

    # ROOT truth → mm/ns/MeV with z-shift
    notes = []
    if ev >= len(vtx) or mu_idx.get(ev) is None:
        notes.append("no-muon-in-truth")
        xr=yr=zr=t_root_ns=E_root_mev=np.nan
        u_root = np.array([np.nan, np.nan, np.nan])
    else:
        vx, vy, vz, vt = vtx[ev]  # meters, seconds
        xr, yr, zr = 1000*vx, 1000*vy, 1000*vz + ZSHIFT_MM
        t_root_ns = vt*1e9
        E_root_mev = 1000*float(p4[ev][mu_idx[ev], 3])   # MeV
        p_root_mev = 1000*np.asarray(p4[ev][mu_idx[ev], 0:3], dtype=float)
        u_root = unit(p_root_mev)

    # residuals
    dx, dy, dz = xg - xr, yg - yr, zg - zr
    rmm, zabs  = (xg**2+yg**2)**0.5, abs(zg)
    in_can     = (rmm < R_CAN_MM) and (zabs < Z_HALF_MM)

    dot = float(np.clip(np.dot(u_g4, u_root), -1.0, 1.0)) if np.isfinite(u_root).all() else np.nan
    dtheta_deg = math.degrees(math.acos(dot)) if np.isfinite(dot) else np.nan

    dE_mev     = (E_g4_mev - E_root_mev) if np.isfinite(E_root_mev) else np.nan
    dE_rel_pct = 100.0*dE_mev/E_root_mev if (np.isfinite(E_root_mev) and E_root_mev!=0) else np.nan

    dt_ns      = (t_g4_ns - t_root_ns) if (np.isfinite(t_g4_ns) and np.isfinite(t_root_ns)) else np.nan

    # pass/fail flags
    vtx_ok = all(np.isfinite([dx,dy,dz])) and (abs(dx)<=TOL_VTX_MM and abs(dy)<=TOL_VTX_MM and abs(dz)<=TOL_VTX_MM)
    ang_ok = (np.isfinite(dtheta_deg) and dtheta_deg<=TOL_ANG_DEG)
    Ene_ok = (np.isfinite(dE_rel_pct) and abs(dE_rel_pct)<=TOL_ERE_PCT)
    tim_ok = (np.isfinite(dt_ns) and abs(dt_ns)<=TOL_DT_NS) if not math.isnan(t_g4_ns) else True

    if not vtx_ok: notes.append("vtx>2mm")
    if not ang_ok: notes.append("ang>1deg")
    if not Ene_ok: notes.append("E>1%")
    if not tim_ok: notes.append("t>1ns")
    if not in_can: notes.append("!inCan")

    rows.append(dict(
        ev=ev,
        # vertex
        dx=dx, dy=dy, dz=dz, r_mm=rmm, z_abs_mm=zabs, inCan=in_can,
        # direction/energy/time
        p_meas_MeV=p_mag_mev,
        E_g4_MeV=E_g4_mev, E_root_MeV=E_root_mev, dE_MeV=dE_mev, dE_over_E_pct=dE_rel_pct,
        t_g4_ns=t_g4_ns, t_root_ns=t_root_ns, dt_ns=dt_ns,
        dtheta_deg=dtheta_deg,
        # pass flags + notes
        vtx_ok=vtx_ok, ang_ok=ang_ok, Ene_ok=Ene_ok, tim_ok=tim_ok,
        PASS=(vtx_ok and ang_ok and Ene_ok and in_can and tim_ok),
        notes=";".join(notes)
    ))

df = pd.DataFrame(rows).sort_values("ev").reset_index(drop=True)

# ------------------ Display & save ------------------
fmt = {
    "dx":"{:.2f}","dy":"{:.2f}","dz":"{:.2f}",
    "r_mm":"{:.1f}","z_abs_mm":"{:.1f}",
    "p_meas_MeV":"{:.1f}",
    "E_g4_MeV":"{:.1f}","E_root_MeV":"{:.1f}","dE_MeV":"{:.2f}","dE_over_E_pct":"{:.3f}",
    "t_g4_ns":"{:.3f}","t_root_ns":"{:.3f}","dt_ns":"{:.3f}",
    "dtheta_deg":"{:.3f}"
}
display(df[[
    "ev","dx","dy","dz","r_mm","z_abs_mm","inCan",
    "dtheta_deg",
    "p_meas_MeV","E_g4_MeV","E_root_MeV","dE_MeV","dE_over_E_pct",
    "t_g4_ns","t_root_ns","dt_ns",
    "PASS","notes"
]].style.format(fmt).apply(lambda s: ['background-color: #eaffea' if v else '' for v in (df["PASS"])]))

passed = bool(df["PASS"].all())
print(f"\nInside can: {int(df['inCan'].sum())}/{len(df)}")
print("Result:", "PASS ✅ (mm/1°/1%/1ns)" if passed else "CHECK ❗ — see 'notes'")

out_dir = "/Users/jingyuanzhang/Desktop/AstroParticle/FLOUNDER/Simulations/GEANT4data/docs/day1"
os.makedirs(out_dir, exist_ok=True)
csv_path = os.path.join(out_dir, "handcheck_summary.csv")
txt_path = os.path.join(out_dir, "handcheck_summary.txt")
df.to_csv(csv_path, index=False)

with open(txt_path, "w") as f:
    f.write("ev  dx  dy  dz  r  |z| inCan  dtheta  |p|  E_g4  E_root  dE  dE/E%  t_g4  t_root  dt  PASS  notes\n")
    for r in rows:
        f.write("{ev:2d} {dx:6.2f} {dy:6.2f} {dz:6.2f} {r_mm:6.1f} {z_abs_mm:6.1f} {inCan!s:>5s} "
                "{dtheta_deg:7.3f} {p_meas_MeV:7.1f} {E_g4_MeV:7.1f} {E_root_MeV:7.1f} {dE_MeV:7.2f} "
                "{dE_over_E_pct:7.3f} {t_g4_ns:7.3f} {t_root_ns:7.3f} {dt_ns:7.3f} {PASS!s:>5s} {notes}\n".format(**r))

print("Saved:", csv_path)
print("Saved:", txt_path)



,ev,dx,dy,dz,r_mm,z_abs_mm,inCan,dtheta_deg,p_meas_MeV,E_g4_MeV,E_root_MeV,dE_MeV,dE_over_E_pct,t_g4_ns,t_root_ns,dt_ns,PASS,notes
0,0,-0.00,0.00,-0.00,1089.4,2757.0,True,0.000,1043.6,1048.9,1048.9,-0.00,-0.000,147.215,147.215,-0.000,True,
1,1,-0.00,-0.00,0.00,1106.4,7402.1,True,0.000,1132.2,1137.1,1137.1,-0.00,-0.000,137.060,137.060,-0.000,True,
2,2,0.00,-0.00,-0.01,621.4,19390.8,True,0.040,3964.2,3965.6,3965.6,-0.00,-0.000,141.242,141.242,-0.000,True,
3,3,-0.00,0.00,-0.00,1286.0,19968.9,True,0.000,1026.1,1031.6,1031.6,0.00,0.000,135.904,135.904,0.000,True,
4,4,-0.00,-0.00,0.00,1484.1,15104.5,True,0.018,23281.9,23282.1,23282.1,-0.00,-0.000,140.093,140.093,0.000,True,
5,5,0.00,0.00,0.02,1162.0,18643.7,True,0.052,1845.6,1848.6,1848.6,0.00,0.000,159.874,159.874,-0.000,True,
6,6,-0.00,-0.00,-0.02,1260.4,19941.0,True,0.034,3327.8,3329.5,3329.5,-0.00,-0.000,136.505,136.505,0.000,True,
7,7,0.00,0.00,-0.01,888.1,16261.0,True,0.000,3945.8,3947.2,3947.2,0.00,0.000,154.916,154.916,-0.000,True,
8,8,-0.00,0.00,-0.03,1193.8,11381.1,True,0.047,632.6,641.4,641.4,-0.00,-0.000,134.890,134.890,-0.000,True,
9,9,-0.00,-0.00,-0.00,997.9,3428.9,True,0.054,3109.1,3110.9,3110.9,0.00,0.000,138.294,138.294,-0.000,True,



Inside can: 10/10
Result: PASS ✅ (mm/1°/1%/1ns)
Saved: /Users/jingyuanzhang/Desktop/AstroParticle/FLOUNDER/Simulations/GEANT4data/docs/day1/handcheck_summary.csv
Saved: /Users/jingyuanzhang/Desktop/AstroParticle/FLOUNDER/Simulations/GEANT4data/docs/day1/handcheck_summary.txt


In [ ]:
import os, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- paths ---
BASE = "/Users/jingyuanzhang/Desktop/AstroParticle/FLOUNDER/Simulations/GEANT4data/docs/day3"
CSV  = f"{BASE}/event_budget.csv"
PLOT_DIR = f"{BASE}/plots"
os.makedirs(PLOT_DIR, exist_ok=True)

# --- load ---
df_raw = pd.read_csv(CSV)

# --- tolerant column resolver (snake_case or camelCase both OK) ---
col_index = {c.lower().replace("_",""): c for c in df_raw.columns}

def pick(*cands):
    for cand in cands:
        key = cand.lower().replace("_","")
        if key in col_index:
            return col_index[key]
    raise KeyError(f"Missing any of columns {cands} in {list(df_raw.columns)}")

c_evt = pick("event","evt","evtid")
c_np  = pick("n_produced","nprod")
c_nw  = pick("n_wall","nwall")
c_nm  = pick("n_pmt","npmt")
c_dt  = pick("first_residual_ns","firstResidualNs","firstdt","dt_first")

# --- normalize types ---
df = pd.DataFrame({
    "event": df_raw[c_evt],
    "nprod": pd.to_numeric(df_raw[c_np], errors="coerce"),
    "nwall": pd.to_numeric(df_raw[c_nw], errors="coerce"),
    "npmt":  pd.to_numeric(df_raw[c_nm], errors="coerce"),
    "first_residual_ns": pd.to_numeric(df_raw[c_dt], errors="coerce"),
})

# --- invariants (warn + show offenders if any) ---
bad_wall = df.loc[df["nwall"] > df["nprod"]]
bad_pmt  = df.loc[df["npmt"]  > df["nprod"]]
if not bad_wall.empty or not bad_pmt.empty:
    print("⚠️ Invariant violation(s) found:")
    if not bad_wall.empty:
        print(" - Nwall > Nprod (showing first 5):")
        display(bad_wall.head())
    if not bad_pmt.empty:
        print(" - Npmt > Nprod (showing first 5):")
        display(bad_pmt.head())
else:
    print("✅ Invariants hold: Nwall ≤ Nprod and Npmt ≤ Nprod")

# Useful ratios
df["wall_frac"] = df["nwall"] / df["nprod"].replace(0, np.nan)

# --- plot 1: histogram of first residuals ---
# Our CSV writes 0.0 if residual is missing; treat ≤0 as missing to avoid bias
x = df["first_residual_ns"].copy()
x = x.replace([-1, np.inf, -np.inf], np.nan)
x = x.where(x > 0)  # drop zeros/negatives that came from "missing"

plt.figure()
plt.hist(x.dropna(), bins=20, range=(0, 1), histtype="stepfilled", color="C0", alpha=0.7)
plt.xlabel("firstΔt (ns)")
plt.ylabel("events")
plt.title("Day-3: first-hit time residual")
plt.tight_layout()
plt.savefig(f"{PLOT_DIR}/firstResidual_hist.png", dpi=200)
plt.close()

# --- plot 2: scatter Nwall vs Nprod (log-log) ---
plt.figure()
plt.scatter(df["nprod"]+1, df["nwall"]+1, s=10)
plt.xscale("log"); plt.yscale("log")
plt.xlabel("Nprod + 1")
plt.ylabel("Nwall + 1")
plt.title("Day-3: wall vs produced")
plt.tight_layout()
plt.savefig(f"{PLOT_DIR}/Nwall_vs_Nprod.png", dpi=200)
plt.close()

# --- text summary ---
summary_path = f"{BASE}/summary.txt"
with open(summary_path, "w") as f:
    f.write(f"events={len(df)}\n")
    finite = x.dropna()
    f.write(f"firstΔt(ns): mean={finite.mean():.4f}  std={finite.std():.4f}  n={finite.size}\n")
    f.write(f"median wall_frac={df['wall_frac'].median():.4f}\n")
    f.write(f"n with Nwall>Nprod={len(bad_wall)}  n with Npmt>Nprod={len(bad_pmt)}\n")

print("✅ QC done →")
print(f" - {PLOT_DIR}/firstResidual_hist.png")
print(f" - {PLOT_DIR}/Nwall_vs_Nprod.png")
print(f" - {summary_path}")

✅ Invariants hold: Nwall ≤ Nprod and Npmt ≤ Nprod
✅ QC done →
 - /Users/jingyuanzhang/Desktop/AstroParticle/FLOUNDER/Simulations/GEANT4data/docs/day3/plots/firstResidual_hist.png
 - /Users/jingyuanzhang/Desktop/AstroParticle/FLOUNDER/Simulations/GEANT4data/docs/day3/plots/Nwall_vs_Nprod.png
 - /Users/jingyuanzhang/Desktop/AstroParticle/FLOUNDER/Simulations/GEANT4data/docs/day3/summary.txt
